In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !rm -R qmc qmc1
    !git clone https://github.com/esgantivar/qmc.git
    !mv qmc qmc1
    !mv qmc1/qmc .
else:
    import sys
    sys.path.insert(0, "../")

rm: cannot remove 'qmc': No such file or directory
rm: cannot remove 'qmc1': No such file or directory
Cloning into 'qmc'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 297 (delta 128), reused 243 (delta 91), pack-reused 0
Receiving objects: 100% (297/297), 16.96 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [2]:
!pip install tensornetwork -q

     |████████████████████████████████| 378kB 9.9MB/s 


In [4]:
import numpy as np
import tensorflow as tf
from qmc.tn.models import DenseCompClassifierQMC
from tensorflow.keras.models import Sequential
from tensornetwork.tn_keras.dense import DenseDecomp

## MNIST

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [6]:
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [7]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[1]) / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[1]) / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((60000, 784), (60000, 10)), ((10000, 784), (10000, 10)))

### DenseDecomp Single Layer

In [8]:
model = Sequential()
model.add(DenseDecomp(10, decomp_size=128, activation='softmax', input_shape=(x_train.shape[1],)))

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_decomp (DenseDecomp)   (None, 10)                101642    
Total params: 101,642
Trainable params: 101,642
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x=x_train, y=y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3617 - accuracy: 0.8972 - val_loss: 0.2998 - val_accuracy: 0.9169
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3016 - accuracy: 0.9166 - val_loss: 0.2925 - val_accuracy: 0.9177
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2903 - accuracy: 0.9193 - val_loss: 0.2816 - val_accuracy: 0.9209
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2838 - accuracy: 0.9211 - val_loss: 0.2911 - val_accuracy: 0.9203
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2777 - accuracy: 0.9232 - val_loss: 0.3011 - val_accuracy: 0.9179
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2753 - accuracy: 0.9247 - val_loss: 0.2847 - val_accuracy: 0.9214
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2721 - accuracy: 0.9238 - val_loss: 0.2822 - val_accuracy:

### DenseCompClassifierQMC

In [14]:
model_qmc = DenseCompClassifierQMC()

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_qmc.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])

In [16]:
history = model_qmc.fit(x_train, 
                    y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 6s 13ms/step - loss: 0.4317 - accuracy: 0.8909 - val_loss: 0.2165 - val_accuracy: 0.9378
Epoch 2/10
469/469 [==============================] - 6s 12ms/step - loss: 0.1884 - accuracy: 0.9463 - val_loss: 0.1668 - val_accuracy: 0.9509
Epoch 3/10
469/469 [==============================] - 6s 12ms/step - loss: 0.1421 - accuracy: 0.9590 - val_loss: 0.1412 - val_accuracy: 0.9582
Epoch 4/10
469/469 [==============================] - 6s 12ms/step - loss: 0.1160 - accuracy: 0.9661 - val_loss: 0.1238 - val_accuracy: 0.9655
Epoch 5/10
469/469 [==============================] - 6s 12ms/step - loss: 0.0999 - accuracy: 0.9702 - val_loss: 0.1142 - val_accuracy: 0.9668
Epoch 6/10
469/469 [==============================] - 6s 12ms/step - loss: 0.0877 - accuracy: 0.9745 - val_loss: 0.1116 - val_accuracy: 0.9678
Epoch 7/10
469/469 [==============================] - 6s 12ms/step - loss: 0.0783 - accuracy: 0.9774 - val_loss: 0.1086 - val_accuracy: 0.9685

## FASHION MNIST

In [17]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

4423680/4422102 [==============================] - 0s 0us/step


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [18]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[1]) / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[1]) / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((60000, 784), (60000, 10)), ((10000, 784), (10000, 10)))

### DenseDecomp Single Layer

In [21]:
model = Sequential()
model.add(DenseDecomp(10, decomp_size=128, activation='softmax', input_shape=(x_train.shape[1],)))

In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])

In [23]:
history = model.fit(x_train, 
                    y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 0.5700 - accuracy: 0.8029 - val_loss: 0.5025 - val_accuracy: 0.8250
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4637 - accuracy: 0.8393 - val_loss: 0.5118 - val_accuracy: 0.8191
Epoch 3/10
469/469 [==============================] - 2s 3ms/step - loss: 0.4435 - accuracy: 0.8468 - val_loss: 0.4921 - val_accuracy: 0.8227
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4297 - accuracy: 0.8511 - val_loss: 0.4690 - val_accuracy: 0.8375
Epoch 5/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4254 - accuracy: 0.8522 - val_loss: 0.4937 - val_accuracy: 0.8305
Epoch 6/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4169 - accuracy: 0.8549 - val_loss: 0.4700 - val_accuracy: 0.8361
Epoch 7/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4149 - accuracy: 0.8557 - val_loss: 0.4709 - val_accuracy: 0.8371
Epoch 

### DenseCompClassifierQMC

In [24]:
model_qmc = DenseCompClassifierQMC()

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_qmc.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])

In [26]:
history = model_qmc.fit(x_train, 
                    y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 6s 13ms/step - loss: 0.5795 - accuracy: 0.8054 - val_loss: 0.4499 - val_accuracy: 0.8393
Epoch 2/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3978 - accuracy: 0.8560 - val_loss: 0.4234 - val_accuracy: 0.8463
Epoch 3/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3640 - accuracy: 0.8674 - val_loss: 0.3913 - val_accuracy: 0.8598
Epoch 4/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3399 - accuracy: 0.8763 - val_loss: 0.3751 - val_accuracy: 0.8670
Epoch 5/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3241 - accuracy: 0.8798 - val_loss: 0.3736 - val_accuracy: 0.8629
Epoch 6/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3111 - accuracy: 0.8845 - val_loss: 0.3721 - val_accuracy: 0.8641
Epoch 7/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3024 - accuracy: 0.8881 - val_loss: 0.3718 - val_accuracy: 0.8657